# Machine Learning: Criando um modelo para predição do preço do ativo ELET3 (Eletrobras) utilizando SVM (Suport Vector Machine)
## Objetivo
Vamos trabalhar com a base histórica de preços das ações do ativo ELET3 (Eletrobrás) buscando treinarmos um modelo capaz de prever altas e baixas futuras do papael de um dia para o outro.

## Libs utilizadas
Abaixo podemos ver todas as biblitecas que vamos utilizar nesse estudo
- Warnings : Apenas para ignorar alguns warnings chatos do pandas de algumas boas praticas que nao estamos nos preocupando nesse momento.
- Pandas: lib que utilizaremos para ler e manipular nossosa dados
- NumPy: é um pacote que suporta arrays e matrizes multidimensionais, e nos ajuda com alguns calculos e manipulações em conjunto com o pandas.
- Scikit-Learn (sklearn) : É uma biblioteca de aprendizado de máquina de código aberto para a linguagem de programação Python.
- Matplotlib: é uma biblioteca de software para criação de gráficos e visualizações de dados em geral.
- Seaborn: é uma biblioteca de visualização de dados Python baseada em matplotlib. Ela fornece uma interface de alto nível para desenhar gráficos estatísticos atraentes e informativos.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#Leitura e manipulacao
import pandas as pd
import numpy as np 
#MachineLeaning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
#Visualizacao de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Lendo e Entendendo nosso dataset
- Time: Data do pregão
- Open: Valor do ativo na abertura do mercado (10:00)
- High: Valor maximo que o ativo alcançou no dia
- Low: Valor minimo que o ativo alcançou no dia
- Close: Valor do ativo no fechamento do mercado (18:00)
- TickVolume:
- Real Volume:
- Next Day Close: Valor do ativo no fechamento do dia seguinte
- Next Day Status Close: Se o valor do fechamento do ativo no dia seguinte for maior ao valor de fechamento do dia atual foi maior , então ocorreu uma alta no preço do ativo, se nao ocorreu uma queda (baixa) no preço do ativo.
- Linear Reg: Calculo de regressão linear simples considerando os ultimos 10 candles

- Aroon é um indicador desenvolvido pelo Dr. Tushar Chande, em 1995, publicado no "Technical Analysis of Stocks & Commodities", em setembro de 1995
    - Arron UP (ap): de 10 dias: mostra quantos dias se passaram desde que o preço registrou sua máxima nos últimos 10 dias.
    - Aroon Down (ad): de 10 dias: mostra quantos dias se passaram desde que o preço registrou sua mínima nos últimos 10 dias.

In [ ]:
df = pd.read_csv('/kaggle/input/eletrobras-base-historica/ELET3.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.tail()

In [ ]:
df.describe()

# Identificando correlação entre as dimensões do dataset

Aqui ulizamos um heatmap da biblioteca seaborn para visualizarmos a correlação entre cada dimensão do nosso dataset. (Quanto mais próximo de 1 mais correlatas proporcionalmente estão as dimensôes analisadas)

In [ ]:
corr = df.corr()
fig, ax = plt.subplots(figsize=(10,10))
ax = sns.heatmap(corr,vmin=-1,vmax=1,center=0,cmap=sns.diverging_palette(20, 220, n=200),square=True,annot=True,linewidths=3,ax=ax,)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45,horizontalalignment='right');

# Separando dados de Treino e Teste
Na imagem acima podemos observar que algumas dimensões de nossos dados estão **Correlacionados de forma Diretamente Proporcional** isso quer dizer por ex que: *quando um valor sobe o outro também sobe proporcionalmente*, ou seja , temos possibilidades de exclusão de dimensões se entendermos que duas dimensões estão trazendo a mesma informação.

- Vamor retirar 'high' e 'low' pois estão totalmente correlacionados com 'open' e 'close' (e em testes anteriores já vi que a exclusão dessas dimensões não degrada a performance do modelo).
- Os valores utilizados para treino e teste serão 70% para treino e consequantemente 30% para teste.



In [ ]:
x = df[['open', 'close',  'real_volume','linear_reg', 'ap', 'ad' ]]
y = df['next_day_close_status']
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.10,  shuffle=False)

# Escalando nossos dados

In [ ]:
scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)
treino_x

# Criando nosso modelo
## SVM
Em 1992, Vapnik e colaboradores propuseram um algoritmo supervisionado para classificação que desde então evoluiu para o que agora é conhecido como Support Vector Machines (SVMs): uma classe de algoritmos para classificação e regressão. Entre as principais inovações desse método estão o uso explícito de otimização convexa, teoria de aprendizagem estatística e funções de kernel. O SVM padrão toma como entrada um conjunto de dados e prediz, para cada entrada dada, qual de duas possíveis classes a entrada faz parte, o que faz do SVM um classificador linear binário não probabilístico.

![](https://external-preview.redd.it/xx3AMQb-TbEin-6T0340d3ykH7VKbpJxDZ16d2kNobw.jpg?auto=webp&s=6ad702bd7dae15ae3bb65c48dabf0bc7fb15f4f2)

para saber mais: http://statweb.stanford.edu/~tibs/sta306bfiles/svmtalk.pdf

In [ ]:
modelo = SVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
print(f'Score do nosso modelo: {round(accuracy_score(teste_y, previsoes) * 100,2)}% ... Não parece uma bom percentual de acerto.')

# Validando Score na Matriz de Confusão

É um tabela que mostra as frequências de classificação para cada classe do modelo. Pegando o exemplo acima, ela vai nos mostrar as frequências:

**Verdadeiro positivo (true positive — TP):** ocorre quando no conjunto real, a classe que estamos buscando foi prevista corretamente. Por exemplo, quando a ação subiu e o modelo previu corretamente que ela subiria.

**Falso positivo (false positive — FP):** ocorre quando no conjunto real, a classe que estamos buscando prever foi prevista incorretamente. Exemplo: a ação não subiu, mas o modelo disse que ela subiria.

**Falso verdadeiro (true negative — TN):** ocorre quando no conjunto real, a classe que não estamos buscando prever foi prevista corretamente. Exemplo: a ação não subiu, e o modelo previu corretamente que ela não subiria.

**Falso negativo (false negative — FN):** ocorre quando no conjunto real, a classe que não estamos buscando prever foi prevista incorretamente. Por exemplo, quando a ação subiu e o modelo previu incorretamente que ela não não subiria.

### Visualizando nossa matriz

In [ ]:
classes = ["alta", "baixa"]
cm = confusion_matrix(teste_y, previsoes,  labels=classes)
cm_display = ConfusionMatrixDisplay(cm, display_labels=classes).plot()

### Entendendo melhor o que ela quer nos dizer

In [ ]:

print('Matriz de confusão do nosso modelo')
print(f'TP: {cm[0][0]} casos que a ação subiu e nosso modelo previu que subiria.')
print(f'FP: {cm[0][1]} casos que a ação não subiu mas o medelo previu que ela subiria.')
print(f'TN: {cm[1][0]} casos que a ação não subiu e o modelo previu corretamente que ela não subirira')
print(f'FN: {cm[1][1]} casos que a ação subiu e o modelo previu incorretamente que ela nao subiria.')

# Validando na Pratica !

Analisando as informações acima podemos ver que mesmo com um score de 65.03 que parece horrivel, podemos ter um desempenho razoável quando pensamos em aplicar as predições:

## Estratégia

Vamos imaginar o seguinte: Cada predição de 'alta' que o nosso modelo nos der, vamos comprar um lote padrão do papel (100 ações) na abertura do mercado e vender no fim do dia.

In [ ]:
df_com_previsoes = raw_teste_x
df_com_previsoes['previsoes'] = previsoes
df_com_previsoes['data'] = df['time'][-teste_y.shape[0]:]
df_com_previsoes['next_close'] = df['next_day_close'][-teste_y.shape[0]:]
df_com_previsoes = df_com_previsoes[df_com_previsoes['previsoes'] == 'alta']
df_com_previsoes['diff'] = df_com_previsoes.apply(lambda x : x['next_close'] - x['close'], axis=1)
df_com_previsoes['diff_accumulated'] = df_com_previsoes['diff'].rolling(min_periods=1, window=df_com_previsoes.shape[0]).sum() * 100
df_com_previsoes['month'] = pd.to_datetime(df_com_previsoes['data'])
df_com_previsoes.head()

In [ ]:
fig, ax = plt.subplots(figsize=(25,7))
sns.set_style("white")
ax = sns.lineplot(data=df_com_previsoes,  x="month", y='diff_accumulated', ax=ax)
ax.tick_params(labelrotation=90)

In [ ]:
df_com_previsoes.head(1)

In [ ]:
df_com_previsoes.tail(1)

# Pequenos Ajustes

In [ ]:
df_com_ajustes = df_com_previsoes.copy()
df_com_ajustes.drop(['diff_accumulated'], axis=1, inplace=True)
df_com_ajustes = df_com_ajustes[df_com_ajustes['linear_reg'] <= df_com_ajustes['close']]
df_com_ajustes = df_com_ajustes[df_com_ajustes['ap'] < 70]
df_com_ajustes = df_com_ajustes[df_com_ajustes['ad'] >= 55]
df_com_ajustes['diff_accumulated'] = df_com_ajustes['diff'].rolling(min_periods=1, window=df_com_ajustes.shape[0]).sum() * 100

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
sns.set_style("white")
ax = sns.lineplot(data=df_com_ajustes,  x="month", y='diff_accumulated', ax=ax)
ax.tick_params(labelrotation=90)

In [ ]:
df_com_ajustes.head(1)

In [ ]:
df_com_ajustes.tail(1)